# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,57.65,72,100,3.44,CL,1706636400
1,1,forked river,39.8398,-74.1901,41.11,69,100,5.99,US,1706636400
2,2,rubtsovsk,51.5000,81.2500,-6.03,97,52,3.13,RU,1706636402
3,3,bethel,41.3712,-73.4140,36.36,66,100,3.44,US,1706636402
4,4,port mathurin,-19.6833,63.4167,82.02,77,100,19.80,MU,1706636403


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .5,
    color = "City"
)



# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

max_temp_condition = (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
wind_speed_condition = city_data_df["Wind Speed"] < 4.5
cloudiness_condition = city_data_df["Cloudiness"] == 0
filtered_cities_df = city_data_df[max_temp_condition & wind_speed_condition & cloudiness_condition]

# Drop any rows with null values

filtered_cities_df=city_data_df.dropna()

# Display sample data

filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,57.65,72,100,3.44,CL,1706636400
1,1,forked river,39.8398,-74.1901,41.11,69,100,5.99,US,1706636400
2,2,rubtsovsk,51.5000,81.2500,-6.03,97,52,3.13,RU,1706636402
3,3,bethel,41.3712,-73.4140,36.36,66,100,3.44,US,1706636402
4,4,port mathurin,-19.6833,63.4167,82.02,77,100,19.80,MU,1706636403


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = filtered_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,puerto natales,-51.7236,-72.4875,57.65,72,100,3.44,CL,1706636400,
1,1,forked river,39.8398,-74.1901,41.11,69,100,5.99,US,1706636400,
2,2,rubtsovsk,51.5000,81.2500,-6.03,97,52,3.13,RU,1706636402,
3,3,bethel,41.3712,-73.4140,36.36,66,100,3.44,US,1706636402,
4,4,port mathurin,-19.6833,63.4167,82.02,77,100,19.80,MU,1706636403,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "sort":"distance",
    "type":"hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius},{latitude},{longitude}"
    params["bias"] = f"point:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: No hotel found
forked river - nearest hotel: No hotel found
rubtsovsk - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
stiens - nearest hotel: No hotel found
kimbe - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
prado - nearest hotel: No hotel found
ugoofaaru - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
xapuri - nearest hotel: No hotel fou

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,puerto natales,-51.7236,-72.4875,57.65,72,100,3.44,CL,1706636400,No hotel found
1,1,forked river,39.8398,-74.1901,41.11,69,100,5.99,US,1706636400,No hotel found
2,2,rubtsovsk,51.5000,81.2500,-6.03,97,52,3.13,RU,1706636402,No hotel found
3,3,bethel,41.3712,-73.4140,36.36,66,100,3.44,US,1706636402,No hotel found
4,4,port mathurin,-19.6833,63.4167,82.02,77,100,19.80,MU,1706636403,No hotel found
6,6,adamstown,-25.0660,-130.1015,73.76,91,100,21.79,PN,1706636404,No hotel found
7,7,talnakh,69.4865,88.3972,1.20,79,99,11.36,RU,1706636405,No hotel found
8,8,papatowai,-46.5619,169.4708,51.98,92,42,10.00,NZ,1706636405,No hotel found
9,9,saipan,15.1355,145.7010,77.70,65,100,17.27,MP,1706636406,No hotel found
10,10,carnarvon,-24.8667,113.6333,78.48,70,15,16.46,AU,1706636406,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    size=100,
    alpha=0.7,
    tiles="CartoLight",
    hover_cols=["City", "Hotel Name", "Country"]
)


# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)